##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# データ増強

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/images/data_augmentation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/images/data_augmentation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table>

## 概要

このチュートリアルでは、データ拡張を説明します。これは、画像の回転といったランダム（ただし現実的）な変換を適用することで、トレーニングセットの多様性を拡大する手法です。

データ拡張を次の 2 つの方法で適用する方法を学習します。

- `tf.keras.layers.Resizing`、`tf.keras.layers.Rescaling`、`tf.keras.layers.RandomFlip`、および `tf.keras.layers.RandomRotation` などの Keras 前処理レイヤーを使用します。
- `tf.image.flip_left_right`、`tf.image.rgb_to_grayscale`、`tf.image.adjust_brightness`、`tf.image.central_crop`、および `tf.image.stateless_random*` などの `tf.image` メソッドを使用します。

## セットアップ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

## データセットをダウンロードする

このチュートリアルでは、[tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers) データセットを使用します。便宜上、[TensorFlow Dataset](https://www.tensorflow.org/datasets) を使用してデータセットをダウンロードします。他のデータインポート方法に関する詳細は、[画像読み込み](https://www.tensorflow.org/tutorials/load_data/images)のチュートリアルをご覧ください。


In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

花のデータセットには 5 つのクラスがあります。

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

データセットから画像を取得し、それを使用してデータ増強を実演してみましょう。

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

## Keras 前処理レイヤーを使用する

### リサイズとリスケール


Keras 前処理レイヤーを使用して、画像を一定の形状にサイズ変更し（`tf.keras.layers.Resizing`）、ピクセル値を再スケールする（`tf.keras.layers.Rescaling`）ことができます。

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

注意: 上記のリスケーリングレイヤーは、ピクセル値を `[0,1]` の範囲に標準化します。代わりに `[-1,1]` を用いる場合には、`tf.keras.layers.Rescaling(1./127.5, offset=-1)` と記述します。


次のようにして、これらのレイヤーを画像に適用した結果を可視化します。 

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

ピクセルが `[0, 1]` の範囲にあることを確認します。

In [ ]:
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

### データ増強

`tf.keras.layers.RandomFlip` や `tf.keras.layers.RandomRotation` などの Keras 前処理レイヤーをデータ拡張に使用することができます。

前処理レイヤーをいくつか作成し、同じ画像に繰り返して適用してみましょう。

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Add the image to a batch
image = tf.expand_dims(image, 0)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

データ拡張には、`tf.keras.layers.RandomContrast`、`tf.keras.layers.RandomCrop`、`tf.keras.layers.RandomZoom` など、様々な前処理レイヤーを使用できます。

### Keras 前処理レイヤーを使用するための 2 つのオプション

これらの前処理レイヤーを使用できる方法には 2 つありませうが、これらには重要なトレードオフが伴います。

#### オプション 1: 前処理レイヤーをモデルの一部にする

In [ ]:
model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  # Rest of your model
])

この場合、2 つの重要なポイントがあります。

- データ増強はデバイス上で他のレイヤーと同期して実行されるため、GPU アクセラレーションの恩恵を受けることができます。

- `model.save`を使用してモデルをエクスポートすると、前処理レイヤーはモデルの残りの部分と一緒に保存されます。後でこのモデルをデプロイする場合、画像は自動的に（レイヤーの設定に従い）標準化されます。これにより、サーバーサイドでロジックを再実装する手間が省けます。

注意: データ拡張はテスト時には非アクティブなので、(`Model.evaluate` や `Model.predict` ではなく)  `Model.fit` への呼び出し時にのみ、入力画像を拡張します。

#### オプション 2: 前処理レイヤーをデータセットに適用する

In [ ]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

このアプローチでは、`Dataset.map` を使用して、拡張画像のバッチを生成するデータセットを作成します。この場合は、

- データ拡張は CPU 上で非同期に行われ、ノンブロッキングです。以下に示すように、`Dataset.prefetch` を使用して GPU 上でのモデルのトレーニングをデータの前処理にオーバーラップさせることができます。
- この場合、`Model.save` を呼び出しても、前処理レイヤーはモデルと一緒にエクスポートされません。保存前にモデルに前処理レイヤーをアタッチするか、サーバー側で前処理レイヤーを再実装する必要があります。トレーニングの後、エクスポートする前に前処理レイヤーをアタッチすることができます。


You can find an example of the first option in the [Image classification](classification.ipynb) tutorial. Let's demonstrate the second option here.

### 前処理レイヤーをデータセットに適用する

前に作成した前処理レイヤーを使用して、トレーニング、検証、テスト用のデータセットを構成します。また、パフォーマンス向上のために、並列読み取りとバッファ付きプリフェッチを使用してデータセットを構成し、I/O がブロックされることなくディスクからバッチを生成できるようにします。（データセットのパフォーマンスに関する詳細は、[tf.data API によるパフォーマンス向上](https://www.tensorflow.org/guide/data_performance)ガイドをご覧ください。）

注意: データ拡張はトレーニングセットのみに適用されます。

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefecting on all datasets
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

### モデルをトレーニングする

完全を期すために、準備したデータセットを使用してモデルをトレーニングします。

[Sequential](https://www.tensorflow.org/guide/keras/sequential_model) モデルは、それぞれに最大プールレイヤー（`tf.keras.layers.MaxPooling2D`）を持つ3つの畳み込みブロック（`tf.keras.layers.Conv2D`）で構成されます。ReLU 活性化関数（`'relu'`）により活性化されたユニットが 128 個ある完全に接続されたレイヤー（`tf.keras.layers.Dense`）があります。このモデルの精度は調整されていません（このチュートリアルの目的は、標準的なアプローチを示すことであるため）。

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

`tf.keras.optimizers.Adam` オプティマイザと`tf.keras.losses.SparseCategoricalCrossentropy` 損失関数を選択します。各トレーニングエポックのトレーニングと検証の精度を表示するには、`Model.compile` に `metrics` 引数を渡します。

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

数エポック、トレーニングします。

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

### カスタムデータ増強

また、カスタムデータ拡張レイヤーを作成することもできます。

このセクションでは、これを行うための 2 つの方法を説明します。

- まず、`tf.keras.layers.Lambda` レイヤーを作成します。簡潔なコードを書くには良い方法です。
- 次に、[subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models) を介して新しいレイヤーを記述します。こうすることで、さらに制御できるようになります。

どちらのレイヤーも、確率に従って、画像の色をランダムに反転します。

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) &lt; p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("off")

次に、[サブクラス化](https://www.tensorflow.org/guide/keras/custom_layers_and_models)してカスタムレイヤーを実装します。

In [ ]:
class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

どちらのレイヤーも、上記 1 と 2 のオプションで説明した使用が可能です。

## tf.image を使用する

上記の Keras 前処理ユーティリティは便利ではありますが、より細かい制御には、`tf.data` や `tf.image` を使用して独自のデータ拡張パイプラインやレイヤーを書くことができます。（また、<a>TensorFlow Addons 画像: 演算</a>および [TensorFlow I/O: 色空間の変換](https://www.tensorflow.org/io/tutorials/colorspace)もご覧ください。）

Since the flowers dataset was previously configured with data augmentation, let's reimport it to start fresh:

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

作業に必要な画像を取得します。

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

Let's use the following function to visualize and compare the original and augmented images side-by-side:

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

### データ増強

#### 画像をフリップする

`tf.image.flip_left_right` を使って、画像を縦方向または横方向に反転します。

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

#### 画像をグレースケールにする

`tf.image.rgb_to_grayscale` を使って、画像をグレースケールにできます。

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

#### 画像の彩度を処理する

`tf.image.adjust_saturation` を使用し、彩度係数を指定して画像の彩度を操作します。

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

#### Change image brightness

`tf.image.adjust_brightness` を使用し、明度係数を指定して画像の明度を変更します。

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

#### 画像を中央でトリミングする

`tf.image.central_crop` を使用して、画像の中央から希望する部分までをトリミングします。

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.5)
visualize(image, cropped)

#### 画像を回転させる

`tf.image.rot90` を使用して、画像を 90 度回転させます。

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

### Random transformations

警告: ランダム画像演算には `tf.image.random*` および `tf.image.stateless_random*` の 2 つのセットがあります。`tf.image.random*` 演算は、TF 1.x の古い RNG を使用するため、使用することは強くお勧めしません。代わりに、このチュートリアルで紹介したランダム画像演算を使用してください。詳細については、[乱数の生成](../../guide/random_numbers.ipynb)を参照してください。

画像にランダムな変換を適用すると、データセットの一般化と拡張にさらに役立ちます。現在の `tf.image` は、次の 8 つのランダム画像演算 (ops) を提供します。

- [`tf.image.stateless_random_brightness`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_brightness)
- [`tf.image.stateless_random_contrast`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_contrast)
- [`tf.image.stateless_random_crop`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_crop)
- [`tf.image.stateless_random_flip_left_right`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_left_right)
- [`tf.image.stateless_random_flip_up_down`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_up_down)
- [`tf.image.stateless_random_hue`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_hue)
- [`tf.image.stateless_random_jpeg_quality`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_jpeg_quality)
- [`tf.image.stateless_random_saturation`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_saturation)

These random image ops are purely functional: the output only depends on the input. This makes them simple to use in high performance, deterministic input pipelines. They require a `seed` value be input each step. Given the same `seed`, they return the same results independent of how many times they are called.

Note: `seed` is a `Tensor` of shape `(2,)`  whose values are any integers.

以降のセクションでは、次のことを行います。

1. ランダム画像演算を使用して画像を変換する例を見る。
2. Demonstrate how to apply random transformations to a training dataset.

#### Randomly change image brightness

`tf.image.stateless_random_brightness` を使用し、明度係数と `seed` を指定して、`image` の明度をランダムに変更します。明度係数は、`[-max_delta, max_delta)` の範囲でランダムに選択され、指定された `seed` に関連付けられます。

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_brightness = tf.image.stateless_random_brightness(
      image, max_delta=0.95, seed=seed)
  visualize(image, stateless_random_brightness)

#### Randomly change image contrast

`tf.image.stateless_random_contrast` を使用し、コントラスト範囲と `seed` を指定して、`image` のコントラストをランダムに変更します。コントラスト範囲は、`[lower, upper]` の間隔でランダムに選択され、指定された `seed` に関連付けられます。

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_contrast = tf.image.stateless_random_contrast(
      image, lower=0.1, upper=0.9, seed=seed)
  visualize(image, stateless_random_contrast)

#### Randomly crop an image

`tf.image.stateless_random_crop` を使用し、ターゲットの `size` と `seed` を指定して `image` をランダムにトリミングします。`image` から切り取られる部分は、ランダムに選択されたオフセットにあり、指定された `seed` に関連付けられています。

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_crop = tf.image.stateless_random_crop(
      image, size=[210, 300, 3], seed=seed)
  visualize(image, stateless_random_crop)

### データ増強をデータセットに適用する

前に説明したように、`Dataset.map` を使用してデータセットにデータ拡張を適用します。

In [ ]:
(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

次に、画像のサイズ変更と再スケーリングのためのユーティリティ関数を定義します。この関数は、データセット内の画像のサイズとスケールを統一するために使用されます。

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image, label

また、画像にランダム変換を適用できる `augment` 関数も定義します。この関数は、次のステップのデータセットで使用されます。

In [ ]:
def augment(image_label, seed):
  image, label = image_label
  image, label = resize_and_rescale(image, label)
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
  # Make a new seed.
  new_seed = tf.random.experimental.stateless_split(seed, num=1)[0, :]
  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  return image, label

#### オプション 1: tf.data.experimental.Counte を使用する

`tf.data.experimental.Counter` オブジェクト (`counter` と呼ぶ) を作成し、`(counter, counter)` を含むデータセットを `zip` します。これにより、データセット内の各画像が、`counter` に基づいて（一意の値形状 `(2,)`）に関連付けられるようになります。これは後で、ランダム変換の `seed` 値として `augment` 関数に渡されます。

In [ ]:
# Create a `Counter` object and `Dataset.zip` it together with the training set.
counter = tf.data.experimental.Counter()
train_ds = tf.data.Dataset.zip((train_datasets, (counter, counter)))

`augment` 関数をトレーニングデータセットにマッピングします。

In [ ]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

#### Option 2: Using tf.random.Generator

- `seed` の初期値で `tf.random.Generator` オブジェクトを作成します。同じジェネレータオブジェクトに `make_seeds` 関数を呼び出すと、必ず新しい一意の `seed` 値が返されます。
- ラッパー関数を 1) `make_seeds` 関数を呼び出し、2) 新たに生成された `seed` 値を `augment` 関数に渡してランダム変換を行うように定義します。

Note: `tf.random.Generator` objects store RNG state in a `tf.Variable`, which means it can be saved as a [checkpoint](../../guide/checkpoint.ipynb) or in a [SavedModel](../../guide/saved_model.ipynb). For more details, please refer to [Random number generation](../../guide/random_numbers.ipynb).

In [ ]:
# Create a generator.
rng = tf.random.Generator.from_seed(123, alg='philox')

In [ ]:
# Create a wrapper function for updating seeds.
def f(x, y):
  seed = rng.make_seeds(2)[0]
  image, label = augment((x, y), seed)
  return image, label

ラッパー関数 `f` をトレーニングデータセットにマッピングし、`resize_and_rescale` 関数を検証セットとテストセットにマッピングします。

In [ ]:
train_ds = (
    train_datasets
    .shuffle(1000)
    .map(f, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

前に示したように、これらのデータセットはモデルのトレーニングに使用が可能です。

## 次のステップ

このチュートリアルでは、Keras 前処理レイヤーと `tf.image` を使用したデータ拡張を説明しました。

- モデル内に前処理レイヤーを含める方法については、[画像の分類](classification.ipynb)チュートリアルをご覧ください。
- 前処理レイヤーでテキストを分類する方法にも関心がある場合は、[基本的なテキスト分類](../keras/text_classification.ipynb)チュートリアルをご覧ください。
- `tf.data` については、こちらの[ガイド](../../guide/data.ipynb)でもさらに詳しく学習できます。パフォーマンスを向上させる入力パイプラインの構成方法については、[こちら](../../guide/data_performance.ipynb)をご覧ください。